In [17]:
from supabase import create_client
import pandas as pd

SUPABASE_URL = "https://qmjpxafyxikhynczflch.supabase.co"
SUPABASE_KEY = "sb_publishable_3yT3_nQMXBfLu27TyCU9UQ_jI57jX4q"

supabase = create_client(SUPABASE_URL, SUPABASE_KEY)


In [18]:
import pandas as pd
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.callbacks import EarlyStopping

In [19]:
RUN_INGESTION = True

In [20]:
CSV_PATH = "berlin-air-quality.csv"

In [21]:
# print(local_df.columns.tolist())

In [22]:
import pandas as pd

local_df = pd.read_csv(CSV_PATH)

# Normalize column names
local_df.columns = (
    local_df.columns
    .str.strip()
    .str.lower()
)

# Drop junk unnamed columns
local_df = local_df.loc[:, ~local_df.columns.str.contains("^unnamed", case=False)]

# Ensure correct types
local_df["date"] = pd.to_datetime(local_df["date"])
local_df = local_df.sort_values("date")

# Force numeric + clean
numeric_cols = ["pm25", "pm10", "no2", "o3"]
for col in numeric_cols:
    local_df[col] = pd.to_numeric(local_df[col], errors="coerce")

# Take last 30 days safely
latest_df = local_df.tail(30).copy()

# Drop rows with invalid numeric data
latest_df = latest_df.dropna(subset=numeric_cols)

latest_df["date"] = latest_df["date"].astype(str)

# Prepare records
records = latest_df[
    ["date", "pm25", "pm10", "no2", "o3"]
].to_dict(orient="records")

# Insert into Supabase
supabase.table("air_quality_raw").insert(records).execute()

print(f"Uploaded {len(records)} rows to air_quality_raw")


Uploaded 27 rows to air_quality_raw


In [23]:
response = supabase.table("air_quality_raw").select("*").execute()
print(len(response.data))
print(response.data[:3])

27
[{'id': '9937a1b2-7003-4d7f-a34a-fd819f3fcc54', 'date': '2025-12-15T00:00:00+00:00', 'pm25': 56.0, 'pm10': 31.0, 'no2': 14.0, 'o3': 3.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}, {'id': '69fd31da-6a57-44e8-8027-cbae9af6af14', 'date': '2025-12-16T00:00:00+00:00', 'pm25': 72.0, 'pm10': 31.0, 'no2': 13.0, 'o3': 2.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}, {'id': '63dfad14-9955-421f-a7ef-d58f1a98491e', 'date': '2025-12-17T00:00:00+00:00', 'pm25': 78.0, 'pm10': 25.0, 'no2': 17.0, 'o3': 9.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}]


In [24]:
def get_last_n_days_from_supabase(supabase, table_name, days=30):
    response = (
        supabase.table(table_name)
        .select("*")
        .order("date")
        .execute()
    )

    df = pd.DataFrame(response.data)
    df["date"] = pd.to_datetime(df["date"])

    last_date = df["date"].max()
    start_date = last_date - pd.Timedelta(days=days)

    return df[(df["date"] > start_date) & (df["date"] < last_date)]


In [25]:
response = supabase.table("air_quality_raw").select("*").execute()
print(len(response.data))
print(response.data[:3])

27
[{'id': '9937a1b2-7003-4d7f-a34a-fd819f3fcc54', 'date': '2025-12-15T00:00:00+00:00', 'pm25': 56.0, 'pm10': 31.0, 'no2': 14.0, 'o3': 3.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}, {'id': '69fd31da-6a57-44e8-8027-cbae9af6af14', 'date': '2025-12-16T00:00:00+00:00', 'pm25': 72.0, 'pm10': 31.0, 'no2': 13.0, 'o3': 2.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}, {'id': '63dfad14-9955-421f-a7ef-d58f1a98491e', 'date': '2025-12-17T00:00:00+00:00', 'pm25': 78.0, 'pm10': 25.0, 'no2': 17.0, 'o3': 9.0, 'created_at': '2026-01-15T16:15:26.990958+00:00'}]


In [26]:
last_30_days_df = get_last_n_days_from_supabase(
    supabase,
    table_name="air_quality_raw",
    days=30
)

In [27]:
last_30_days_df.columns = last_30_days_df.columns.str.strip()
print(last_30_days_df.head(30))

cols = ['pm25', 'pm10', 'no2', 'o3']
last_30_days_df[cols] = last_30_days_df[cols].apply(pd.to_numeric, errors='coerce')

print(last_30_days_df.info())


                                      id                      date  pm25  \
1   69fd31da-6a57-44e8-8027-cbae9af6af14 2025-12-16 00:00:00+00:00  72.0   
2   63dfad14-9955-421f-a7ef-d58f1a98491e 2025-12-17 00:00:00+00:00  78.0   
3   b71aa1c4-387f-480b-985f-88e2e3777f55 2025-12-18 00:00:00+00:00  65.0   
4   1a4d6dc1-deae-42e1-867e-7275203653ea 2025-12-19 00:00:00+00:00  56.0   
5   bf1fea30-ed2e-4d05-9072-4b57bbb3905c 2025-12-20 00:00:00+00:00  66.0   
6   3058ba30-7560-466c-997b-eb7da65a0712 2025-12-21 00:00:00+00:00  73.0   
7   13ad2e78-ea94-4980-baf6-4438d93b355c 2025-12-22 00:00:00+00:00  73.0   
8   96222557-cf4b-4472-bd8d-d9b78ba785ac 2025-12-23 00:00:00+00:00  66.0   
9   c28716b0-5057-45e4-bdab-fa49d013fdde 2025-12-24 00:00:00+00:00  44.0   
10  41c3c810-62cc-4a99-9af4-eecf92fe25fe 2025-12-25 00:00:00+00:00  84.0   
11  bd59fae6-bdf2-41ab-b094-fe25773df3e7 2025-12-26 00:00:00+00:00  94.0   
12  ed396ec5-2071-4093-b36c-7a32380ae6b0 2025-12-27 00:00:00+00:00  57.0   
13  2e5e5536

In [30]:
cols = ["pm25", "pm10", "no2", "o3"]

last_30_days_df[cols] = last_30_days_df[cols].interpolate(method="linear")

last_30_days_df[cols] = (
    last_30_days_df[cols]
    .ffill()
    .bfill()
)

print(last_30_days_df.info())
print(last_30_days_df.head(30))


<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 1 to 25
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          25 non-null     object             
 1   date        25 non-null     datetime64[ns, UTC]
 2   pm25        25 non-null     float64            
 3   pm10        25 non-null     float64            
 4   no2         25 non-null     float64            
 5   o3          25 non-null     float64            
 6   created_at  25 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(4), object(2)
memory usage: 1.6+ KB
None
                                      id                      date  pm25  \
1   69fd31da-6a57-44e8-8027-cbae9af6af14 2025-12-16 00:00:00+00:00  72.0   
2   63dfad14-9955-421f-a7ef-d58f1a98491e 2025-12-17 00:00:00+00:00  78.0   
3   b71aa1c4-387f-480b-985f-88e2e3777f55 2025-12-18 00:00:00+00:00  65.0   
4   1a4d6dc1-deae-42e1-867e-7275203653ea 

In [32]:
missing_cols = last_30_days_df.columns[last_30_days_df.isnull().any()]
print("Columns with Missing Values:")
print(missing_cols)

Columns with Missing Values:
Index([], dtype='object')


In [33]:
print(last_30_days_df[last_30_days_df.isna().any(axis=1)])

Empty DataFrame
Columns: [id, date, pm25, pm10, no2, o3, created_at]
Index: []


In [34]:
# Replace '-' with NaN and forward fill
last_30_days_df.replace('-', np.nan, inplace=True)
last_30_days_df.ffill(inplace=True)
last_30_days_df.info()

<class 'pandas.core.frame.DataFrame'>
Index: 25 entries, 1 to 25
Data columns (total 7 columns):
 #   Column      Non-Null Count  Dtype              
---  ------      --------------  -----              
 0   id          25 non-null     object             
 1   date        25 non-null     datetime64[ns, UTC]
 2   pm25        25 non-null     float64            
 3   pm10        25 non-null     float64            
 4   no2         25 non-null     float64            
 5   o3          25 non-null     float64            
 6   created_at  25 non-null     object             
dtypes: datetime64[ns, UTC](1), float64(4), object(2)
memory usage: 1.6+ KB


In [35]:
print(last_30_days_df.head(30))

                                      id                      date  pm25  \
1   69fd31da-6a57-44e8-8027-cbae9af6af14 2025-12-16 00:00:00+00:00  72.0   
2   63dfad14-9955-421f-a7ef-d58f1a98491e 2025-12-17 00:00:00+00:00  78.0   
3   b71aa1c4-387f-480b-985f-88e2e3777f55 2025-12-18 00:00:00+00:00  65.0   
4   1a4d6dc1-deae-42e1-867e-7275203653ea 2025-12-19 00:00:00+00:00  56.0   
5   bf1fea30-ed2e-4d05-9072-4b57bbb3905c 2025-12-20 00:00:00+00:00  66.0   
6   3058ba30-7560-466c-997b-eb7da65a0712 2025-12-21 00:00:00+00:00  73.0   
7   13ad2e78-ea94-4980-baf6-4438d93b355c 2025-12-22 00:00:00+00:00  73.0   
8   96222557-cf4b-4472-bd8d-d9b78ba785ac 2025-12-23 00:00:00+00:00  66.0   
9   c28716b0-5057-45e4-bdab-fa49d013fdde 2025-12-24 00:00:00+00:00  44.0   
10  41c3c810-62cc-4a99-9af4-eecf92fe25fe 2025-12-25 00:00:00+00:00  84.0   
11  bd59fae6-bdf2-41ab-b094-fe25773df3e7 2025-12-26 00:00:00+00:00  94.0   
12  ed396ec5-2071-4093-b36c-7a32380ae6b0 2025-12-27 00:00:00+00:00  57.0   
13  2e5e5536

In [36]:
# Select Features (Pollutant Columns)
features = ['pm25', 'pm10', 'no2', 'o3']
data = last_30_days_df[features]

# Handle Missing Values (Using Simple Interpolation)
# This fills gaps using a linear trend between known values.
data = data.interpolate(method='linear')

# Verify no more missing data
print("\nMissing values after interpolation:")
print(data.isnull().sum())
print("\nProcessed Data Head:")
print(data.head())


Missing values after interpolation:
pm25    0
pm10    0
no2     0
o3      0
dtype: int64

Processed Data Head:
   pm25  pm10   no2    o3
1  72.0  31.0  13.0   2.0
2  78.0  25.0  17.0   9.0
3  65.0  20.0  17.0   7.0
4  56.0  25.0  18.0  10.0
5  66.0  26.0   7.0   8.0


In [37]:
# Scaling and Sequence Generation

# Scaling
scaler = MinMaxScaler(feature_range=(0, 1))
scaled_data = scaler.fit_transform(data)

# Define Sequence Parameters
N_IN = 7      # Lookback window (7 days)
N_OUT = 2     # Prediction horizon (2 days)
N_FEATURES = len(features) # 4 pollutants

# Function to Create Sequences (Sliding Window)
def create_sequences(data, n_in, n_out):
    X, y = [], []
    # Loop from the starting point up to the end minus the total length of the sequence
    for i in range(len(data) - n_in - n_out + 1):
        # Input sequence (7 days)
        end_ix = i + n_in
        X.append(data[i:end_ix, :])

        # Output sequence (2 days starting right after the input ends)
        out_ix = end_ix + n_out
        y.append(data[end_ix:out_ix, :])

    return np.array(X), np.array(y)

# Create the sequences
X, y = create_sequences(scaled_data, N_IN, N_OUT)

print(f"\nShape of Input (X): (Samples, Lookback Days, Features) -> {X.shape}")
print(f"Shape of Output (y): (Samples, Prediction Horizon, Features) -> {y.shape}")

# Chronological Train/Test Split (80% Train, 20% Test)
# Time series data must be split chronologically.
train_size = int(len(X) * 0.8)
X_train, X_test = X[:train_size], X[train_size:]
y_train, y_test = y[:train_size], y[train_size:]

print(f"Train/Test Split: {len(X_train)} training samples, {len(X_test)} test samples.")


Shape of Input (X): (Samples, Lookback Days, Features) -> (17, 7, 4)
Shape of Output (y): (Samples, Prediction Horizon, Features) -> (17, 2, 4)
Train/Test Split: 13 training samples, 4 test samples.


In [38]:
# Build, Train, and Evaluate the LSTM Model

#  Build the Model
model = Sequential()

# LSTM Layer: 50 units, returns sequences for the next layer
model.add(LSTM(50, activation='relu', input_shape=(N_IN, N_FEATURES)))

# Output Layer: Dense layer to predict the 4 features (pollutants) for the 2 days (N_OUT)
# flatten the output (N_OUT * N_FEATURES = 8 values)
model.add(Dense(N_OUT * N_FEATURES))
model.compile(optimizer='adam', loss='mse')

# Display Model Summary
print("\nModel Summary:")
model.summary()

# Training the Model
# EarlyStopping prevents overfitting by stopping training if validation loss doesn't improve.
callback = EarlyStopping(monitor='val_loss', patience=10, verbose=1)

history = model.fit(X_train, y_train.reshape(X_train.shape[0], -1), # Reshape y_train for the model
                    epochs=100,
                    batch_size=32,
                    verbose=1,
                    validation_split=0.1, # Use 10% of training data for validation
                    callbacks=[callback])

# Evaluate Model on the Test Set
test_loss = model.evaluate(X_test, y_test.reshape(X_test.shape[0], -1), verbose=0)
print(f"\nTest Set Mean Squared Error (MSE): {test_loss:.4f}")

# Make Predictions on Test Set
y_pred_scaled = model.predict(X_test)

# Reshape predictions and actuals back to (Samples, N_OUT, N_FEATURES)
y_pred_scaled = y_pred_scaled.reshape(y_test.shape)

# Inverse Transform (Rescale to original values)
# To inverse transform (must first flatten the 3D arrays to 2D)
y_test_original = scaler.inverse_transform(y_test.reshape(-1, N_FEATURES))
y_pred_original = scaler.inverse_transform(y_pred_scaled.reshape(-1, N_FEATURES))

# Reshape back to 3D for comparison plots
y_test_original = y_test_original.reshape(y_test.shape)
y_pred_original = y_pred_original.reshape(y_test.shape)

d:\lifestyle-tracker-ds\venv\Lib\site-packages\keras\src\layers\rnn\rnn.py:199: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(**kwargs)



Model Summary:


Model: "sequential"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ lstm (LSTM)                     │ (None, 50)             │        11,000 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 8)              │           408 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 11,408 (44.56 KB)

 Trainable params: 11,408 (44.56 KB)

 Non-trainable params: 0 (0.00 B)

Epoch 1/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step - loss: 0.3129 - val_loss: 0.4011
Epoch 2/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.3068 - val_loss: 0.3924
Epoch 3/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 85ms/step - loss: 0.3007 - val_loss: 0.3835
Epoch 4/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 84ms/step - loss: 0.2947 - val_loss: 0.3744
Epoch 5/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 76ms/step - loss: 0.2886 - val_loss: 0.3652
Epoch 6/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 77ms/step - loss: 0.2823 - val_loss: 0.3558
Epoch 7/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 74ms/step - loss: 0.2760 - val_loss: 0.3459
Epoch 8/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 83ms/step - loss: 0.2695 - val_loss: 0.3357
Epoch 9/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.2629 - val_loss: 0.3251
Epoch 10/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 88ms/step - loss: 0.2559 - val_loss: 0.3143
Epoch 11/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 81ms/step - loss: 0.2488 - val_loss: 0.3032
Epoch 12/100
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 86ms/step - loss: 0.2415 - val_loss

In [41]:
# Final 2-Day Forecast

# Prepare the Final Input
# Take the last N_IN (7) days from the scaled dataset
last_7_days_scaled = scaled_data[-N_IN:]

# Reshape the input to match model's expected input shape: (1, N_IN, N_FEATURES)
X_input = last_7_days_scaled.reshape(1, N_IN, N_FEATURES)

# Generate the Forecast
forecast_scaled = model.predict(X_input)

# Reshape the forecast (1, N_OUT * N_FEATURES) to (N_OUT, N_FEATURES) for inverse scaling
forecast_scaled = forecast_scaled.reshape(N_OUT, N_FEATURES)

# Inverse Transform the Forecast to Original Units
final_forecast_original = scaler.inverse_transform(forecast_scaled)

#date colum last_30_days
last_30_days_df['date'] = pd.to_datetime(last_30_days_df['date'])
last_30_days_df = last_30_days_df.set_index('date').sort_index()


# Create a Forecast DataFrame
last_date = last_30_days_df.index[-1]
forecast_dates = pd.date_range(start=last_date + pd.Timedelta(days=1), periods=N_OUT, freq='D')

forecast_df = pd.DataFrame(final_forecast_original,
                           index=forecast_dates,
                           columns=features)


print("\n\n#####################################################")
print("## ✅ 2-DAY AIR QUALITY FORECAST (NEXT 48 HOURS) ✅ ##")
print("#####################################################")
print(forecast_df)
print("-----------------------------------------------------")

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 188ms/step


#####################################################
## ✅ 2-DAY AIR QUALITY FORECAST (NEXT 48 HOURS) ✅ ##
#####################################################
                                pm25       pm10        no2         o3
2026-01-14 00:00:00+00:00  63.789726  22.957777  13.238660  17.546480
2026-01-15 00:00:00+00:00  65.523972  22.244024  15.025309  20.777824
-----------------------------------------------------


In [42]:
from datetime import timedelta

last_known_date = last_30_days_df.index.max()
 # from training data

forecast_df = forecast_df.copy()
forecast_df["forecast_date"] = [
    last_known_date + timedelta(days=i + 1)
    for i in range(len(forecast_df))
]


In [43]:
# AQI CALCULATION (POST-PROCESSING, NO MODEL CHANGE) ---

# AQI breakpoint tables (CPCB – India)
aqi_breakpoints = {
    "pm25": [(0,30,0,50),(31,60,51,100),(61,90,101,200),(91,120,201,300),(121,250,301,400),(251,500,401,500)],
    "pm10": [(0,50,0,50),(51,100,51,100),(101,250,101,200),(251,350,201,300),(351,430,301,400),(431,600,401,500)],
    "no2":  [(0,40,0,50),(41,80,51,100),(81,180,101,200),(181,280,201,300),(281,400,301,400),(401,1000,401,500)],
    "o3":   [(0,50,0,50),(51,100,51,100),(101,168,101,200),(169,208,201,300),(209,748,301,400)]
}

def calculate_sub_aqi(pollutant, value):
    for bp_lo, bp_hi, i_lo, i_hi in aqi_breakpoints[pollutant]:
        if bp_lo <= value <= bp_hi:
            return ((i_hi - i_lo)/(bp_hi - bp_lo)) * (value - bp_lo) + i_lo
    return aqi_breakpoints[pollutant][-1][3]  # max AQI for that pollutant


# Calculate AQI for each predicted day
aqi_values = []
aqi_categories = []

for _, row in forecast_df.iterrows():
    sub_indices = [
        calculate_sub_aqi('pm25', row['pm25']),
        calculate_sub_aqi('pm10', row['pm10']),
        calculate_sub_aqi('no2', row['no2']),
        calculate_sub_aqi('o3', row['o3'])
    ]
    final_aqi = int(max(sub_indices))
    aqi_values.append(final_aqi)

    if final_aqi <= 50:
        aqi_categories.append("Good")
    elif final_aqi <= 100:
        aqi_categories.append("Satisfactory")
    elif final_aqi <= 200:
        aqi_categories.append("Moderate")
    elif final_aqi <= 300:
        aqi_categories.append("Poor")
    elif final_aqi <= 400:
        aqi_categories.append("Very Poor")
    else:
        aqi_categories.append("Severe")

# Add AQI to forecast table
forecast_df["AQI"] = aqi_values
forecast_df["AQI_Category"] = aqi_categories

print("## FINAL AQI FORECAST (NEXT 2 DAYS) ##")
print("#############################################")
print(forecast_df)
print("---------------------------------------------")


## FINAL AQI FORECAST (NEXT 2 DAYS) ##
#############################################
                                pm25       pm10        no2         o3  \
2026-01-14 00:00:00+00:00  63.789726  22.957777  13.238660  17.546480   
2026-01-15 00:00:00+00:00  65.523972  22.244024  15.025309  20.777824   

                                      forecast_date  AQI AQI_Category  
2026-01-14 00:00:00+00:00 2026-01-14 00:00:00+00:00  110     Moderate  
2026-01-15 00:00:00+00:00 2026-01-15 00:00:00+00:00  116     Moderate  
---------------------------------------------


In [44]:
forecast_df = forecast_df.rename(columns={"forecast_date": "date"})

In [45]:
forecast_df["date"] = forecast_df["date"].astype(str)

In [46]:
forecast_records = forecast_df[[
    "date",
    "pm25",
    "pm10",
    "no2",
    "o3"
]].to_dict(orient="records")


In [47]:
# Append forecasted rows into raw air quality table
supabase.table("air_quality_raw").insert(forecast_records).execute()


APIResponse(data=[{'id': 'c2bd71c1-e6ec-4ee0-9ca1-02f70df95a60', 'date': '2026-01-14T00:00:00+00:00', 'pm25': 63.78972625732422, 'pm10': 22.95777702331543, 'no2': 13.238659858703613, 'o3': 17.546480178833008, 'created_at': '2026-01-15T16:24:00.073027+00:00'}, {'id': '9d4c2e94-3fab-450b-ac2d-5091f32c0a85', 'date': '2026-01-15T00:00:00+00:00', 'pm25': 65.52397155761719, 'pm10': 22.2440242767334, 'no2': 15.025308609008789, 'o3': 20.77782440185547, 'created_at': '2026-01-15T16:24:00.073027+00:00'}], count=None)

In [48]:
print(forecast_df.index[-1])

2026-01-15 00:00:00+00:00


In [49]:
records = []

for forecast_date, row in forecast_df.iterrows():
    records.append({
        "forecast_date": forecast_date.isoformat(),
        "pm25": float(row["pm25"]),
        "pm10": float(row["pm10"]),
        "no2": float(row["no2"]),
        "o3": float(row["o3"]),
        "aqi": int(row["AQI"]),
        "aqi_category": row["AQI_Category"]
    })

supabase.table("air_quality_forecast").insert(records).execute()

print("Pushed rows:", len(records))


Pushed rows: 2
